In [3]:
# Importando todas as bibliotecas necessárias para o projeto
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from rouge_score import rouge_scorer
import bert_score


##Caso seja necessário instalar alguma biblioteca:
##%pip install rouge_score bert_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
# Define a lista de stopwords em português (palavras comuns que serão removidas no pré-processamento)
stop_words = nltk.corpus.stopwords.words('portuguese')

# Define o stemmer para inglês (caso trabalhe com textos em inglês)
stemmer_en = PorterStemmer()

# Define o stemmer para português (reduz palavras ao seu radical no idioma português)
stemmer_pt = nltk.stem.RSLPStemmer()

def preprocess_text(text, stop_words = stop_words, stemmer = stemmer_pt):
    # Transforma tudo para minúsucla
    s = str(text).lower()

    # Tokeniza o texto (quebra em palavras)
    tokens = word_tokenize(s)

    # Remove stopwords, pontuações, caracteres especiais e números isolados
    filtered_tokens = [word for word in tokens if word not in stop_words and word.isalnum() and not word.isdigit()]

    # Aplica stemming (reduz palavras para seus radicais)
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    return stemmed_tokens

In [6]:
# Baixar o tokenizer
# ⬆️ Enviar o arquivo Resumos.csv manualmente no Colab (ícone de pasta > upload)
# Depois leia:
df = pd.read_csv("Resumos.csv")

# Verificação das colunas
if "Original" not in df.columns or "Resumo" not in df.columns:
    raise ValueError("CSV precisa ter as colunas 'Original' e 'Resumo'.")

# Lista de resultados
resultados = []

# Avaliar cada linha
for _, row in df.iterrows():
    ref = str(row["Original"])
    res = str(row["Resumo"])

    # ROUGE
    ref_tokens = ' '.join(preprocess_text(ref))
    res_tokens = ' '.join(preprocess_text(res))
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    rouge_scores = scorer.score(ref_tokens, res_tokens)
    rouge1 = rouge_scores['rouge1'].fmeasure

    # BERTScore
    _, _, F1 = bert_score.score([res], [ref], lang="pt", verbose=False)

    # Coleta
    resultados.append({
        "Texto Original": ref,
        "Resumo": res,
        "ROUGE-1": rouge1,
        "BERTScore-F1": F1[0].item()
    })

# DataFrame final
df_metricas = pd.DataFrame(resultados)

# Imprimir DataFrame em arquivo para Analise das Métricas
df_metricas.to_csv('TCC_Resultados_Metricas_Resumo.csv', index=False,encoding='utf-8-sig')

# Mostrar tabela
print(df_metricas)


,Texto Original,Resumo,ROUGE-1,BERTScore-F1
0,Uma narrativa feita por Apolodoro acerca do ba...,Platão explora o conceito de Eros em 'O Banque...,0.301887,0.695334
1,"Ambientado no cárcere de Atenas, o Fédon mostr...","Sócrates, em seus últimos momentos, oferece um...",0.227848,0.748286
2,"Em ""Totem e tabu"", Sigmund Freud conduziu sua ...",Freud analisa as origens da cultura através do...,0.361111,0.792135
3,É um pequeno romance publicado em 1864. Consid...,Um homem amargurado e solitário questiona a vi...,0.106061,0.682701
4,Brás Cubas está morto. Mas isso não o impede d...,"Um relato pós-morte de Brás Cubas, repleto de ...",0.168421,0.704857
5,Bentinho e Capitu são criados juntos e se apai...,"Na obra, Bentinho e Capitu se apaixonam na ado...",0.187500,0.685737
6,Esta história de coragem e triunfo pessoal é a...,Um pescador enfrenta a solidão e a adversidade...,0.062500,0.629411
7,O clássico que valeu o Prêmio Pulitzer a John ...,Um clássico sobre a luta da família Joad contr...,0.200000,0.693852
8,"Em um cortiço decadente e superlotado, localiz...","Em um cortiço decadente do Rio de Janeiro, a a...",0.320988,0.762601
9,Uma obra fundamental da historiografia brasile...,"Clássico da historiografia brasileira, 'Raízes...",0.595745,0.842825
